---
title: "Cortex Forecast Example"
date: "08/20/2024"
format:
    html:
        code-fold: False
        output: False
---

# Overview

In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
#| hide
from nbdev.showdoc import *

# Example Useage

In [1]:
#| skip
from cortex_forecast.forecast import SnowflakeMLForecast
from snowflake.snowpark.version import VERSION
import os

In [21]:
#| skip
forecast_model = SnowflakeMLForecast(
    config_file='./cortex_forecast/files/yaml/taxi_forecast_config.yaml',
    connection_config={
        'user': os.getenv('SNOWFLAKE_USER'),
        'password': os.getenv('SNOWFLAKE_PASSWORD'),
        'account': os.getenv('SNOWFLAKE_ACCOUNT'),
        'database': 'CORTEX',
        'warehouse': 'CORTEX_WH',
        'schema': 'DEV',
        'role': 'CORTEX_USER_ROLE'  # Use the desired role
    }
)

snowflake_environment = forecast_model.session.sql('SELECT current_user(), current_version()').collect()
snowpark_version = VERSION
print('\nConnection Established with the following parameters:')
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(forecast_model.session.get_current_role()))
print('Database                    : {}'.format(forecast_model.session.get_current_database()))
print('Schema                      : {}'.format(forecast_model.session.get_current_schema()))
print('Warehouse                   : {}'.format(forecast_model.session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0], snowpark_version[1], snowpark_version[2]))


Connection Established with the following parameters:
User                        : JD_SERVICE_ACCOUNT_ADMIN
Role                        : "CORTEX_USER_ROLE"
Database                    : "CORTEX"
Schema                      : "DEV"
Warehouse                   : "CORTEX_WH"
Snowflake version           : 8.31.1
Snowpark for Python version : 1.19.0


# Make Data Sets

> TODO: Come in and create the data assets, but for now we are skipping directly to a training table and a inference table. 

Create these tables by following along the [snowflake QuickStart](https://quickstarts.snowflake.com/guide/geo-for-machine-learning/index.html#2)

## Preview Training Table

In [22]:
forecast_model.session.sql(f"""
    SELECT * FROM {forecast_model.config['input_data']['table']} LIMIT 10
""").to_pandas()

,PICKUP_TIME,H3,PICKUPS,SCHOOL_HOLIDAY,PUBLIC_HOLIDAY,SPORT_EVENT
0,2014-07-04 20:00:00,882a10721bfffff,700,school-holidays,"holiday, holiday-national",None
1,2014-07-04 23:00:00,882a100d35fffff,318,school-holidays,"holiday, holiday-national",None
2,2014-09-01 19:00:00,882a100f53fffff,244,school-holidays,"holiday, holiday-national",None
3,2014-12-25 09:00:00,882a10721bfffff,158,school-holidays,"holiday, holiday-christian, holiday-national, ...",None
4,2014-07-04 12:00:00,882a103b03fffff,119,school-holidays,"holiday, holiday-national",None
5,2014-09-06 15:00:00,882a100d63fffff,1024,school-holidays,None,None
6,2014-09-05 16:00:00,882a1072cdfffff,866,school-holidays,None,None
7,2014-08-20 08:00:00,882a100d35fffff,131,school-holidays,None,None
8,2014-08-09 10:00:00,882a100e25fffff,151,school-holidays,None,None
9,2014-08-11 03:00:00,882a10089bfffff,476,school-holidays,None,None


### Inference Table Preview

In [23]:
forecast_model.session.sql(f"""
    SELECT * FROM {forecast_model.config['forecast_config']['table']} LIMIT 10
""").to_pandas()

,H3,PICKUP_TIME,SCHOOL_HOLIDAY,PUBLIC_HOLIDAY,SPORT_EVENT
0,882a100d67fffff,2015-06-05 20:00:00,None,None,None
1,882a100d67fffff,2015-06-05 17:00:00,None,None,None
2,882a100d67fffff,2015-06-10 03:00:00,None,None,None
3,882a1072c1fffff,2015-06-10 18:00:00,None,None,None
4,882a1072cbfffff,2015-06-09 15:00:00,None,None,None
5,882a100d6bfffff,2015-06-09 04:00:00,None,None,None
6,882a100899fffff,2015-06-10 04:00:00,None,None,None
7,882a1008b3fffff,2015-06-11 08:00:00,None,None,None
8,882a100883fffff,2015-06-07 04:00:00,None,None,None
9,882a100d21fffff,2015-06-11 04:00:00,None,None,None


# Run Forecast

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from cortex_forecast.forecast import SnowflakeMLForecast
from snowflake.snowpark.version import VERSION
import os


In [3]:
# Run Forecast Default to 60 Days To Speed Up Training Full Takes ~6 Minutes
forecast_model = SnowflakeMLForecast(
    config_file='./cortex_forecast/files/yaml/taxi_forecast_config.yaml',
    connection_config={
        'user': os.getenv('SNOWFLAKE_USER'),
        'password': os.getenv('SNOWFLAKE_PASSWORD'),
        'account': os.getenv('SNOWFLAKE_ACCOUNT'),
        'database': 'CORTEX',
        'warehouse': 'CORTEX_WH',
        'schema': 'DEV',
        'role': 'CORTEX_USER_ROLE'  # Use the desired role
    }
)
forecast_data = forecast_model.create_and_run_forecast()
forecast_data

Attempting to create tag: environment
Tag 'environment' already exists.
Attempting to create tag: team
Tag 'team' already exists.
Step 1/4: Creating training table...
Generated SQL:

        CREATE OR REPLACE TRANSIENT TABLE my_taxi_forecast_model_20240821_awzgk_train AS
        SELECT TO_TIMESTAMP_NTZ(pickup_time) AS pickup_time, pickups AS target, * EXCLUDE (pickup_time, pickups)
        FROM ny_taxi_rides_h3_train
        WHERE TO_TIMESTAMP_NTZ(pickup_time) < DATEADD(day, -30, CURRENT_DATE());
Step 2/4: Creating forecast model...
Generated SQL:

        CREATE OR REPLACE SNOWFLAKE.ML.FORECAST my_taxi_forecast_model_20240821_awzgk(
            INPUT_DATA => SYSTEM$REFERENCE('table', 'ny_taxi_rides_h3_train'),
            TIMESTAMP_COLNAME => 'pickup_time',
            TARGET_COLNAME => 'pickups',
        SERIES_COLNAME => 'h3',
CONFIG_OBJECT => {'on_error': 'skip', 'evaluate': TRUE, 'evaluation_config': {'n_splits': 2, 'gap': 0, 'prediction_interval': 0.95}}) WITH TAG (environment = 

,H3,PICKUP_TIME,FORECAST,LOWER_BOUND,UPPER_BOUND
0,882a100e25fffff,2015-06-04 21:00:00,17.939327,0.000000,189.139774
1,882a100e25fffff,2015-06-04 22:00:00,7.744896,0.000000,178.945343
2,882a100e25fffff,2015-06-04 23:00:00,0.000000,0.000000,169.385932
3,882a100f57fffff,2015-06-04 23:00:00,2.523767,0.000000,75.479801
4,882a100891fffff,2015-06-05 00:00:00,25.719744,0.000000,199.473400
...,...,...,...,...,...
7386,882a100d6bfffff,2015-06-11 23:00:00,88.055720,0.000000,248.903834
7387,882a1072cdfffff,2015-06-11 23:00:00,222.313270,0.000000,473.742683
7388,882a1072c7fffff,2015-06-11 23:00:00,82.984636,0.000000,231.615926
7389,882a107219fffff,2015-06-11 23:00:00,156.401550,65.890229,246.912871


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()